# Importo Pacchetti

In [1]:
import numpy as np
import os
import importlib
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy.optimize import curve_fit
from scipy.integrate import solve_ivp
from scipy.integrate import odeint
from numba import jit
import nolds
import sys
sys.path.insert(0,'./fortran_package/') # carico la directory con i pacchetti
import maps
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
import plotly.graph_objects as go
import plotly.io as pio
import ComputeLE
from numba import jit, njit
import time

In [2]:
importlib.reload(maps)

<module 'maps' from '/home/edo/Università/DNL/Seminario_Finale/Code/./fortran_package/maps.cpython-38-x86_64-linux-gnu.so'>

# Simulo la dinamica

## Esponenti di Lyapunov

In [3]:
@njit
def LV(t, val, a, r):
    """
    Questa funzione definisce gli incrementi secondo il formato utile a 'solve_ivp' di scykit_learn.
    """
    ss = [np.sum((np.ones(len(val)) - val) * a[i]) for i in range(len(a))]
    diff = [r[i] * val[i] * ss[i]  for i in range(len(r))]
    return np.array(diff)

@njit
def RK4(f, x, t1, t2, a, r):
    """
    Fourth-order, 4-step RK routine.
    Returns the step, i.e. approximation to the integral.
    If x is defined at time t_1, then stim should be an array of
    stimulus values at times t_1, (t_1+t_2)/2, and t_2 (i.e. at t1 and t2, as
    well as at the midpoint).
    Alternatively, stim may be a function pointer.
    """
    tmid = (t1 + t2)/2
    dt = t2 - t1

    K1 = f(t1, x, a, r)
    K2 = f(tmid, x + dt*K1/2, a, r)
    K3 = f(tmid, x + dt*K2/2, a, r)
    K4 = f(t2, x + dt*K3, a, r)

    return dt * (K1/2 + K2 + K3 + K4/2) / 3

@njit
def motion(f, t, x, a, r):
    for i,(t1,t2) in enumerate(zip(t[:-1], t[1:])):
        x[i+1] = x[i] + RK4(f, x[i], t1, t2, a, r)
    return x

In [4]:
@jit
def sim_mu(mu, init = np.array([0.3, 0.3, 0.3]), n = 20000, tot_t = 2000, term = False):
    t = np.linspace(0, tot_t, n)
    # Parameters
    a = np.array([[0.5,   0.5, 0.1],
                  [-0.5, -0.1, 0.1],
                  [  mu,  0.1, 0.1]])
    r = np.array([1, 1, 1])
    param = [a, r]
    x = np.ones((n, 3))*init
    if term: 
        x = motion(LV, t, x, a, r)
        x[0] = x[-1]
    x = motion(LV, t, x, a, r)
    xx = x[:,0]
    yy = x[:,1]
    zz = x[:,2]
    return xx, yy, zz

In [21]:
def writefiles(n_files = 10):
    mus = np.linspace(1.2, 1.65, n_files) 
    init = np.array([1., 1., 0.99])
    for i, mu in enumerate(mus):
        xx, yy, zz = sim_mu(mu, init = init,  n = 50000, tot_t=500, term = False)
        np.savetxt(f'data/frames/sim{i}.txt', (xx, yy, zz))

In [22]:
start = time.time()
writefiles(100)
print(time.time()-start)

21.501333236694336
